In [1]:
%pip install zstandard
%pip install psutil
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

data_files = "https://huggingface.co/datasets/qualis2006/PUBMED_title_abstracts_2020_baseline/resolve/main/PUBMED_title_abstracts_2020_baseline.jsonl.zst"
pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
pubmed_dataset

Loading dataset shards:   0%|          | 0/49 [00:00<?, ?it/s]

Dataset({
    features: ['meta', 'text'],
    num_rows: 17722096
})

In [7]:
# -> 17 722 096 lignes et 2 colonnes

In [3]:
pubmed_dataset[0]

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [4]:
import psutil

print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

print(f"Number of files in dataset : {pubmed_dataset.dataset_size}")
size_gb = pubmed_dataset.dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

RAM used: 13651.81 MB
Number of files in dataset : 24453015916
Dataset size (cache file) : 22.77 GB


In [5]:
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

#time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
#print(
    #f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "#f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s")

In [6]:
pubmed_dataset_streamed = load_dataset(
    "json", data_files=data_files, split="train", streaming=True
)

In [7]:
next(iter(pubmed_dataset_streamed))

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x["text"]))
next(iter(tokenized_dataset))

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [9]:
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
next(iter(shuffled_dataset))

{'meta': {'pmid': 1675166, 'language': 'ita'},
 'text': '[Benzodiazepine withdrawal syndrome].\nBenzodiazepines (BDZ) are widely prescribed in clinical practice for many pathological conditions, because of their anxiolytic, sedative, myorelaxant and anticonvulsant properties. The effectiveness, specificity and rapidity of action, the few side effects and the virtual absence of toxicity, have contributed to the widespread use of these compounds. In the last decade, however, the attitude towards BDZ has greatly changed, due to growing awareness and concern about dependence liability, withdrawal phenomena, and long-term side effects. Withdrawal symptoms have been singled out and specified in the contest of a well-defined syndrome with foreseeable onset, duration and remission. Psychic and physical symptoms and disorders of sensory perception can be observed. These manifestations can be suppressed by resuming treatment. The symptomatic and developmental aspects of BDZ withdrawal syndrome a

In [10]:
dataset_head = pubmed_dataset_streamed.take(5)
list(dataset_head)

[{'meta': {'pmid': 1673585, 'language': 'eng'},
  'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was

In [ ]:
from huggingface_hub import login

login() 

In [15]:
pile_dataset = load_dataset("bigcode/the-stack-v2", split="train", streaming=True)
#next(iter(pile_dataset["train"]))
pile_dataset

Resolving data files:   0%|          | 0/917 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/917 [00:00<?, ?it/s]

IterableDataset({
    features: ['blob_id', 'directory_id', 'path', 'content_id', 'detected_licenses', 'license_type', 'repo_name', 'snapshot_id', 'revision_id', 'branch_name', 'visit_date', 'revision_date', 'committer_date', 'github_id', 'star_events_count', 'fork_events_count', 'gha_license_id', 'gha_event_created_at', 'gha_created_at', 'gha_language', 'src_encoding', 'language', 'is_vendor', 'is_generated', 'length_bytes', 'extension'],
    num_shards: 917
})